<a href="https://colab.research.google.com/github/AIJuand/AI/blob/main/Modelo1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo 1 (Generacion de Poemas)**
Andrey Duvan Rincon Torres

---

In [ ]:
pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 3.2 MB/s 
     |████████████████████████████████| 419 kB 44.7 MB/s 
     |████████████████████████████████| 596 kB 14.2 MB/s 
     |████████████████████████████████| 140 kB 54.7 MB/s 
     |████████████████████████████████| 1.1 MB 23.9 MB/s 
     |████████████████████████████████| 144 kB 45.1 MB/s 
     |████████████████████████████████| 271 kB 49.6 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Basicas
import pandas as pd
import numpy as np
# Graficas
import plotly.express as plx
import plotly.graph_objects as go
# Pytorch
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchmetrics.functional import accuracy
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# Texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
def poem_to_string(poem):
  return f'\n{poem["title"]}\n{poem["author"]}\n{poem["content"]}'
def poem_sequence_to_string(poem_sequence):
    poem_stringified = tokenizer.sequences_to_texts([poem_sequence])[0]
    print(poem_stringified)
CELoss = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AVAIL_GPUS = min(1, torch.cuda.device_count())

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Datos**

In [ ]:
url = 'https://raw.githubusercontent.com/andreamorgar/poesIA/master/data/poems.csv'
poems_df = pd.read_csv(url)
poems_df = poems_df.dropna()

In [ ]:
# Filtrar poemas grandes
poems_df['string'] = poems_df.apply(lambda row: f'\n{row["title"]}\n\n{row["content"]}', axis=1)
poems_df['length'] = poems_df.string.map(len)
MAX_POEM_LENGTH=1000
poems_filtered = poems_df[poems_df.length<MAX_POEM_LENGTH]
_ , poems_filtered = train_test_split(poems_filtered, test_size = 0.05 ,shuffle=True)
poems_filtered

,author,content,title,string,length
1480,Federico García Lorca,\nLa muchacha dorada \nse bañaba en el agua \n...,Casida de la muchacha dorada,\nCasida de la muchacha dorada\n\n\nLa muchach...,602
2692,Rubén Darío,"\nYa de un corintio templo cincela una metopa,...",J.J. Palma,\nJ.J. Palma\n\n\nYa de un corintio templo cin...,630
2687,Toni García Arias,Te miré a los ojos\ny tú\nrespondiste a mi mir...,Mirada,\nMirada\n\nTe miré a los ojos\ny tú\nrespondi...,572
4798,Luis de Góngora,"\n\nPor niñear, un picarillo tierno,\nHurón de...","Por niñear, un picarillo tierno","\nPor niñear, un picarillo tierno\n\n\n\nPor n...",575
728,Jesús Hilario Tundidor,"\n\nFácil, en la meseta castellana\nes el ciel...",PAÍS DEL ÁGUILA,"\nPAÍS DEL ÁGUILA\n\n\n\nFácil, en la meseta c...",692
...,...,...,...,...,...
2592,Julia de Burgos,"\nParece mar, el cielo\ndonde me he recostado ...",Azul de tierra en tí,"\nAzul de tierra en tí\n\n\nParece mar, el cie...",901
2844,Luis de Góngora,"\n\n¡Oh qué bien que baila Gil,\nCon las mozas...",La chacona a las sonajas,\nLa chacona a las sonajas\n\n\n\n¡Oh qué bien...,763
3885,Carlos Edmundo de Ory,\n\nEn el fondo de ti vuela la mariposa\nperso...,NIHILISMO,\nNIHILISMO\n\n\n\nEn el fondo de ti vuela la ...,411
4658,Ana Istarú,\n\nSobre tu frente\nlos lirios mal heridos.\n...,SOBRE TU FRENTE,\nSOBRE TU FRENTE\n\n\n\nSobre tu frente\nlos ...,639


In [ ]:
print(list(poems_filtered['string'])[0])


Casida de la muchacha dorada


La muchacha dorada 
se bañaba en el agua 
y el agua se doraba. 
Las algas y las ramas 
en sombra la asombraban 
y el ruiseñor cantaba 
por la muchacha blanca. 
Vino la noche clara, 
turbia de plata mata, 
con peladas montañas 
bajo la brisa parda. 
La muchacha mojada 
era blanca en el agua, 
y el agua, llamarada. 
Vino el alba sin mancha, 
con mil caras de vaca, 
yerta y amortajada 
con heladas guirnaldas. 
La muchacha de lágrimas 
se bañaba entre llamas, 
y el ruiseñor lloraba 
con las alas quemadas. 
La muchacha dorada 
era una blanca garza 
y el agua la doraba.


# **Vocabulary**

In [ ]:
poems_string=poems_filtered.string
STOP_SIGN = '␣'
tokenizer = Tokenizer(char_level=True, filters='', lower=False, split='')
tokenizer.fit_on_texts([STOP_SIGN])
tokenizer.fit_on_texts(poems_string)
total_words = len(tokenizer.word_index) + 1
dataset_vectorized = tokenizer.texts_to_sequences(poems_string)
max_sequence_len = max([len(x) for x in dataset_vectorized])
data_train, data_val = train_test_split(dataset_vectorized, test_size = 0.1, random_state = 0,shuffle=True)

In [ ]:
def collate_batch(batch):
    input_sequences = []
    for line in batch:
      token_list = line
      for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label_list = torch.tensor(label, dtype=torch.int64)
    text_list = torch.tensor(predictors, dtype=torch.int64)
    return text_list, label_list
x_train, y_train = collate_batch(data_train)
x_val, y_val = collate_batch(data_val)

In [ ]:
for i in range(10):
  print(tokenizer.sequences_to_texts([list(x_val[i][x_val[i] != 0].detach().numpy())])[0] + '-->' + tokenizer.sequences_to_texts([[y_val.detach().numpy()[i]]])[0])


-->E

 E-->l

 E l--> 

 E l  -->m

 E l   m-->a

 E l   m a-->r

 E l   m a r--> 

 E l   m a r  -->t

 E l   m a r   t-->ú

 E l   m a r   t ú--> 


## **Modelo**

In [ ]:
class Model(nn.Module):
  # creamos la estructura de la red
  def __init__(self):
      super(Model,self).__init__()
      # Embeding de las palabras
      self.embedding = nn.Embedding(total_words, 100)
      # Red Bidireccional
      self.lstm_1 = nn.LSTM(100, 150, 1, batch_first=True, dropout = 0.2 , bidirectional = True)
      # red LSTM
      self.lstm_2 = nn.LSTM(300,100,1, batch_first=True)
      # red perceptron
      #-- self.linear_1 = nn.Linear(100,1605)
      self.linear_2 = nn.Linear(100,total_words)
      #-- self.relu = nn.ReLU()
      self.sofmax = nn.Softmax()
  # definimos el comportamiento de las capas
  def forward(self, x):
      batch_size, channelsn = x.size()
      x = self.embedding(x)
      # layer LSTM bidirectional
      out, (h_n, c_n) = self.lstm_1(x)
      # layer LSTM
      out, (h_n, c_n) = self.lstm_2(out)
      out = torch.squeeze(h_n)
      # capa de salida
      #-- out = self.relu(self.linear_1(out))
      out = self.sofmax(self.linear_2(out))
      return out

## **Datos**

In [ ]:
# Clase de los datos
class DataModule(pl.LightningDataModule):
  # Definimos un tamaño de lote en la calse
  def __init__(self, batch_size = 32):
      super(DataModule,self).__init__()
      self.batch_size = batch_size
  # Definimos el tratamiento de los datos
  def setup(self, stage=None):
    self.train_dataset = TensorDataset(x_train,y_train)
    self.val_dataset = TensorDataset(x_train,y_train)
  # Iterable de entrenamiento
  def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch_size)
  # Iterable de validacion
  def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch_size)

## **Entrenamiento**

In [ ]:
class Train(pl.LightningModule):
    # creamos la estructura de la red
    def __init__(self,model):
        super().__init__()
        self.model = model
   # Paso de entrenamiento
    def training_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss
    # Paso de validacion
    def validation_step(self, batch, batch_idx):
        loss,acc = self._shared_eval_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss
    # Funcion para evaluar el modelo y la perdida
    def _shared_eval_step(self,batch,batch_idx):
        x, y  = batch
        y_hat = self.model(x)
        y_hat = y_hat.to(device)
        y = y.type(torch.LongTensor).to(device)
        loss = CELoss(y_hat, y)
        acc = accuracy(y_hat, y)
        return loss, acc
    # Configuracion del optimizador
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters())

##  Ajustar el modelo

In [ ]:
data_module = DataModule() # Ejecutamos modulo de datos
torch.manual_seed(0)
model = Model() # Ejecutamos modelo
trainer = pl.Trainer(max_epochs=50, progress_bar_refresh_rate=20,gpus=AVAIL_GPUS) # Lamamos el entrenador
task = Train(model)
trainer.fit(task,data_module)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logg

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# guardar modelo
PATH = './checkpoint.pt'
torch.save(model.state_dict(), PATH)

In [ ]:
# Tablero de resultados
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

# **Generacion de texto**

In [ ]:
seed_text = "Who are you, so too cruel?"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.forward(torch.tensor(token_list.astype(np.float32))).detach().numpy().transpose()[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)